# Semantic baseline, 2
Данный ноутбук продолждает решение на USE с использованием улучшенного парсера, новых полей и расширения запросов.

In [1]:
import tensorflow_hub as hub
import tensorflow_text
import numpy as np
from sklearn import metrics
import tensorflow as tf
import os
import pickle
from tqdm.notebook import tqdm
from collections import defaultdict, OrderedDict
import requests

Скачиваем модель.

In [2]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [3]:
PATH = "/Users/michelle/data/nn_text-relevance-competition-ir-1-ts-fall-2020/"

Читаем запросы и sample, переводим sample в словарь запрос-документы.

In [9]:
with open(PATH + "queries.numerate.txt", "r") as f:
    queries = f.read().split("\n")

In [10]:
with open(PATH + "sample.technosphere.ir1.textrelevance.submission.txt", "r") as f:
    sample = f.read().split("\n")

In [11]:
query_docid = defaultdict(list)
for item in sample[1:]:
    if item:
        qid = item.split(",")[0]
        docid = item.split(",")[1]
        query_docid[qid].append(docid)

Читаем эмбединги запросов

In [12]:
with open(os.path.join(PATH, "queries_embeddings"), "rb") as f:
    queries_embeddings = pickle.load(f)

# Headers

Читаем поля headers, полученные с помощью парсера с улучшенной обработкой пунктукции. Для каждого документа выбираем имеющийся header с максимальным порядком.

In [48]:
listdir = os.listdir(os.path.join(PATH, "parsed_punc_content"))

headers = defaultdict(str)
bad_files = []
for file in tqdm(listdir, total=len(listdir)):
    try:
        docid = file.split("_")[-1].split(".")[0]
        with open(os.path.join(PATH, "parsed_punc_content", file), "rb") as f:
            for header in pickle.load(f)[3]:
                if header != "":
                    headers[docid] = header
                    break
                    
    except:
        bad_files.append(file)

In [49]:
with open(os.path.join(PATH, "headers_punc.pkl"), "wb") as f:
    pickle.dump(headers, f)

In [51]:
!mkdir /Users/michelle/data/nn_text-relevance-competition-ir-1-ts-fall-2020/headers_punc_embeddings

С помощью USE считаем эмбединги headers

In [71]:
bad_queries = []
for query, docids in tqdm(query_docid.items(), total=len(query_docid)):
    try:
        headers_tmp = [headers[docid][:1000] for docid in docids]
        headers_emb_tmp = embed(headers_tmp)
        with open(os.path.join(PATH, "headers_punc_embeddings", f"{query}.pkl"), "wb") as f:
            pickle.dump(headers_emb_tmp, f)
        
    except:
        print(query)
        bad_queries.append(query)

Для каждой пары запрос-документ считаем косинусы между эмебдингом запроса и эмбедингом заголовка или headerа. Складываем получившиеся величины с весами 0.7 и 0.3 соответственно.

In [72]:
ranged_query_docid = {}
for query, docids in tqdm(query_docid.items(), total=len(query_docid)):
    with open(os.path.join(PATH, f"title_embeddings/{query}.pkl"), "rb") as f: # titles with punc
        data_title = pickle.load(f)
    
    with open(os.path.join(PATH, f"headers_punc_embeddings/{query}.pkl"), "rb") as f: # headers without punc
        data_headers = pickle.load(f)
        
    title_score = metrics.pairwise.cosine_distances(tf.reshape(queries_embeddings[int(query) - 1], (1, -1)), data_title)[0]
    headers_score = metrics.pairwise.cosine_distances(tf.reshape(queries_embeddings[int(query) - 1], (1, -1)), data_headers)[0]
    
    ranged_query_docid[query] = np.array(docids)[np.argsort(0.7 * title_score + 0.3 * headers_score)]

In [73]:
for query in ranged_query_docid:
    ranged_query_docid[query] = list(ranged_query_docid[query])

In [75]:
submit = ['QueryId,DocumentId']
for i in range(1, len(ranged_query_docid) + 1):
    docids = ranged_query_docid[str(i)]
    for docid in docids:
        submit.append(f"{i},{docid}")
        
with open(PATH + "submit_10_4.txt", "w") as f:
    f.write("\n".join(submit))

Далее делаем всё то же самое, только с эмбедингами для расширенных запросов.

In [84]:
with open(os.path.join(PATH, "queries_all_large.pkl"), "rb") as f:
    queries = pickle.load(f)

In [77]:
queries_to_map = []

for query in queries[:-1]:
    queries_to_map.append(query.split("\t")[1])

In [79]:
queries_embeddings = embed(queries_to_map)

In [80]:
with open(os.path.join(PATH, "queries_embeddings_ext.pkl"), "wb") as f:
    pickle.dump(queries_embeddings, f)

In [81]:
ranged_query_docid = {}
for query, docids in tqdm(query_docid.items(), total=len(query_docid)):
    with open(os.path.join(PATH, f"title_embeddings/{query}.pkl"), "rb") as f: # titles with punc
        data_title = pickle.load(f)
    
    with open(os.path.join(PATH, f"headers_punc_embeddings/{query}.pkl"), "rb") as f: # headers without punc
        data_headers = pickle.load(f)
        
    title_score = metrics.pairwise.cosine_distances(tf.reshape(queries_embeddings[int(query) - 1], (1, -1)), data_title)[0]
    headers_score = metrics.pairwise.cosine_distances(tf.reshape(queries_embeddings[int(query) - 1], (1, -1)), data_headers)[0]
    
    ranged_query_docid[query] = np.array(docids)[np.argsort(0.7 * title_score + 0.3 * headers_score)]

In [82]:
for query in ranged_query_docid:
    ranged_query_docid[query] = list(ranged_query_docid[query])

In [86]:
submit = ['QueryId,DocumentId']
for i in range(1, len(ranged_query_docid) + 1):
    docids = ranged_query_docid[str(i)]
    for docid in docids:
        submit.append(f"{i},{docid}")
        
with open(PATH + "submit_10_5.txt", "w") as f:
    f.write("\n".join(submit))